In [1]:
%matplotlib widget
import matplotlib
from matplotlib import pyplot as plt
import xarray as xr
import pandas as pd
import numpy as np
import tqdm
import pickle
import pint
import pint_xarray
import cftime
u = pint.UnitRegistry()

xarray DataArray:

* all fixed variables are dimensions and coordinates
* space is "allocated" for one data point of observation
* each observation gets their own DataArray

A collection of DataArrays with shared dimensions/coords, i.e. where the coords of each individual DataArray is a subset of the shared coords, can form a Dataset.

Because not all DataArrays may share dimensions/coords, there is possibly a need for a container, which is not yet defined.

There are, again questions around representation of units.

# representation
* load all example data sets into the data structure at once
* measure memory use
* export the data sets to CSV
* reimport from CSV
* export the data sets to some kind of native storage format
* reimport from the native storage format

## read in the data

In [2]:
with open(f'mem/fao_emi_total.pck', 'rb') as fd:
    fao_emi_total_pd = pickle.load(fd)

fao_emi_total_pd.index = (fao_emi_total_pd.index.set_levels(fao_emi_total_pd.index.levels[0].astype('str'), level=0)
                                                .set_levels(fao_emi_total_pd.index.levels[2].astype('str'), level=2)
                                                .set_levels(fao_emi_total_pd.index.levels[3].astype('str'), level=3)
                         )
fao_emi_total = xr.DataArray.from_series(fao_emi_total_pd.pint.magnitude)
fao_emi_total.attrs['units'] = 'Gg'

In [3]:
with open(f'mem/lak_emi_energy.pck', 'rb') as fd:
    lak_emi_energy_pd = pickle.load(fd)

lak_emi_energy_pd.index = (lak_emi_energy_pd.index.set_levels(lak_emi_energy_pd.index.levels[0].astype('str'), level=0)
                                                  .set_levels(lak_emi_energy_pd.index.levels[0].astype('str'), level=0)
                          )
lak_emi_energy = xr.DataArray.from_series(lak_emi_energy_pd.pint.magnitude)
lak_emi_energy.attrs['units'] = 'kilometric_ton'

In [4]:
with open(f'mem/cmip_conc_co2.pck', 'rb') as fd:
    cmip_conc_co2_pd = pickle.load(fd)
    
cmip_conc_co2_pd.index = cmip_conc_co2_pd.index.set_levels(cmip_conc_co2_pd.index.levels[0].astype('str'), level=0)
cmip_conc_co2 = xr.DataArray.from_series(cmip_conc_co2_pd.pint.magnitude)

from itertools import product
from cftime import DatetimeGregorian
cmip_conc_co2["Date"] = xr.CFTimeIndex([DatetimeGregorian(year, month, 1) for year, month in product([-1] + list(range(1, 2015)), range(1, 13))])
cmip_conc_co2.attrs['units'] = 'dimensionless'

In [5]:
with open(f'mem/prm_emi.pck', 'rb') as fd:
    prm_emi_pd = pickle.load(fd)

prm_emi_pd.index = (prm_emi_pd.index.set_levels(prm_emi_pd.index.levels[0].astype('str'), level=0)
                                    .set_levels(prm_emi_pd.index.levels[1].astype('str'), level=1)
                                    .set_levels(prm_emi_pd.index.levels[3].astype('str'), level=3)
                                    .set_levels(prm_emi_pd.index.levels[4].astype('str'), level=4)
                   )
prm_emi = xr.DataArray.from_series(prm_emi_pd.pint.magnitude)
prm_emi.attrs['units'] = 'Gg'

In [6]:
ds = prm_emi.to_dataset('Entity')
dss = {}
for da in ds:
    dss[da] = ds[da].dropna('Category', how='all')
    dss[da].attrs['units'] = 'Gg'

prm_emi_d = dss

In [7]:
with open(f'mem/primap_tables.pck', 'rb') as fd:
    primap_tables_pd = pickle.load(fd)

In [8]:
primap_arrays = {}
for key, (meta, dt) in tqdm.tqdm_notebook(primap_tables_pd.items()):
    primap_arrays[key] = xr.DataArray.from_series(dt.pint.magnitude)
    primap_arrays[key].attrs.update(meta)
    primap_arrays[key].attrs['units'] = str(dt.pint.units)

In [9]:
primap_arrays['BC_IPC0_TOTAL_NET_HISTORY_BUR2IPCC2006I']

<xarray.DataArray 'BC' (Area: 1, Date: 26)>
array([[ 91.37982154,  95.67522655,  95.36063434,  98.26888393,
         97.17830312, 102.33396701, 102.14369614, 108.48512431,
        119.0760673 , 102.8574417 , 102.7730411 ,  97.95328695,
         95.84174417,  95.3765566 ,  92.87245032, 110.99820393,
        110.43118983, 119.09241552, 133.02842217, 121.67774323,
        115.22365168, 117.64828036, 114.53743313, 120.34479469,
        113.13672179, 112.24474635]])
Coordinates:
  * Area     (Area) object 'MEX'
  * Date     (Date) datetime64[ns] 1990-01-01 1991-01-01 ... 2015-01-01
Attributes:
    category:       IPC0
    class:          TOTAL
    code:           BC_IPC0_TOTAL_NET_HISTORY_BUR2IPCC2006I
    datatype:       CountryYearMatrix
    descr:          GHG data submitted to the UNFCCC by Parties in the Bianni...
    name_category:  NationalTotal
    note:           Read in by JG
    scenario:       HISTORY
    source:         BUR2IPCC2006I
    subsource:      [['MEX_BUR2_IPCC2006_2018.csv']]
    tablekind:      raw
    type:           NET
    units:          gigagram

In [10]:
def primap_to_dataset(source):
    Entities = {}
    Categories = []
    Classes = []
    Types = []
    Scenarios = []
    Areas = set()
    Dates = set()
    Units = {}
    for key, da in primap_arrays.items():
        if da.attrs['source'] != source:
            continue
        Entity, Category, Class, Type, Scenario, Source = key.split('_')
        for v, l in ((Category, Categories), (Class, Classes), (Type, Types), (Scenario, Scenarios)):
            if v not in l:
                l.append(v)
        
        da.attrs['Entity'] = Entity
        da.attrs['Category'] = Category
        da.attrs['Class'] = Class
        da.attrs['Type'] = Type
        da.attrs['Scenario'] = Scenario
        da.attrs['Source'] = Source
        
        if Entity not in Entities:
            Entities[Entity] = []
            
        Entities[Entity].append(da)
        
        if Entity not in Units:
            Units[Entity] = []
        
        if da.attrs['units'] not in Units[Entity]:
            Units[Entity].append(da.attrs['units'])
                
        Areas = Areas.union(set(da.Area.values))
        Dates = Dates.union(set(da.Date.values))
    
    ndas = {}
    for Entity, das in Entities.items():

        convert_to_gigagram = False
        if len(Units[Entity]) > 1 and 'gigagram' in Units[Entity]:  # TODO: deal with incompatible units
            Units[Entity] = ['gigagram']
            convert_to_gigagram = True
        elif len(Units[Entity]) > 1:
            raise ValueError(str(Units[Entity]))
    
        dim_sources = (Categories, Classes, Types, Scenarios, Areas, Dates)
        shape = [len(x) for x in dim_sources]
        coords = [np.array(sorted(x)) for x in dim_sources]
        em = np.empty(shape, dtype=np.float)
        em[:] = np.nan
        nda = xr.DataArray(data=em, coords=coords, dims=['Category', 'Class', 'Type', 'Scenario', 'Area', 'Date'], name=source)

        for da in das:
            at = da.attrs
            sel = {key: at[key] for key in ('Scenario', 'Category', 'Class', 'Type')}
            
            if convert_to_gigagram:
                db = da.pint.quantify().pint.to('gigagram').pint.dequantify().broadcast_like(nda.loc[sel])
            else:
                db = da.broadcast_like(nda.loc[sel])
            
            nda.loc[sel] = db

        nda.attrs['units'] = Units[Entity][0]
        
        ndas[Entity] = nda
        
    return xr.Dataset(ndas)

def dataset_to_reduced_dict(ds, dropna_dims):
    d = {}
    for key in ds:
        s = ds[key]
        for dropna_dim in dropna_dims:
            s = s.dropna(dropna_dim, how='all')
        d[key] = s
    return d

In [11]:
def normalize_dates_to_cftime_index(dates):
    if isinstance(dates.values[0], int):
        cftimes = [cftime.DatetimeJulian(year=x, month=1, day=1) for x in dates.values]
    elif isinstance(dates.values[0], pd.Period):
        cftimes = [cftime.DatetimeJulian(year=x.year, month=x.month, day=x.day) for x in dates.values]
    elif isinstance(dates.values[0], np.datetime64):
        cftimes = [cftime.DatetimeJulian(year=x.year, month=x.month, day=x.day) for x in pd.DatetimeIndex(dates.values)]
    elif isinstance(dates.values[0], cftime.datetime):
        cftimes = [cftime.DatetimeJulian(year=x.year, month=x.month, day=x.day) for x in dates.values]
    return xr.CFTimeIndex(cftimes)

In [12]:
for key, da in primap_arrays.items():
    if da.attrs['source'] != 'MPD2018P':
        continue
    da['Date'] = normalize_dates_to_cftime_index(da['Date'])

In [13]:
sources = set([x.attrs['source'] for x in primap_arrays.values()])
primap_sources = {}
for source in tqdm.tqdm_notebook(sources):
    primap_sources[source] = primap_to_dataset(source)

KeyboardInterrupt: 

## Measure memory use

In [16]:
!mkdir -p xr/mem/

import pickle

for fpath, obj in (
    ('fao_emi_total', fao_emi_total),
    ('lak_emi_energy', lak_emi_energy),
    ('cmip_conc_co2', cmip_conc_co2),
    ('prm_emi', prm_emi), 
    ('primap_arrays', primap_arrays),
    ('primap_sources', primap_sources),
):
    with open(f'xr/mem/{fpath}.pck', 'wb') as fd:
        pickle.dump(obj, fd, -1)

!ls -lah xr/mem/
!ls -lah mem/

total 1,2G
drwxrwxr-x 2 pflueger pflueger 4,0K Aug 31 17:48 .
drwxrwxr-x 4 pflueger pflueger 4,0K Aug 31 18:40 ..
-rw-rw-r-- 1 pflueger pflueger 1,3M Sep  1 12:05 cmip_conc_co2.pck
-rw-rw-r-- 1 pflueger pflueger 7,5M Sep  1 12:05 fao_emi_total.pck
-rw-rw-r-- 1 pflueger pflueger  28K Sep  1 12:05 lak_emi_energy.pck
-rw-rw-r-- 1 pflueger pflueger 134M Sep  1 12:05 primap_arrays.pck
-rw-rw-r-- 1 pflueger pflueger 888M Sep  1 12:05 primap_sources.pck
-rw-rw-r-- 1 pflueger pflueger 137M Sep  1 12:05 prm_emi.pck
total 674M
drwxrwxr-x 2 pflueger pflueger 4,0K Aug 27 10:55 .
drwxrwxr-x 8 pflueger pflueger 4,0K Sep  1 11:28 ..
-rw-rw-r-- 1 pflueger pflueger 1,2M Aug 27 18:49 cmip_conc_co2.pck
-rw-rw-r-- 1 pflueger pflueger 1,2M Aug 27 18:53 cmip_conc_co2_wo.pck
-rw-rw-r-- 1 pflueger pflueger 8,8M Aug 27 18:49 fao_emi_total.pck
-rw-rw-r-- 1 pflueger pflueger 9,7M Aug 27 18:53 fao_emi_total_wo.pck
-rw-rw-r-- 1 pflueger pflueger  40K Aug 27 18:49 lak_emi_energy.pck
-rw-rw-r-- 1 pflueger pflueger  

## I/O
* export the data sets to CSV
* reimport from CSV
* export the data sets to some kind of native storage format
* reimport from the native storage format

In [18]:
# via pandas like:
#fao_emi_total.to_dataframe().dropna(how='all').to_csv('test.csv')

In [18]:
!mkdir -p xr/nc/

for fpath, obj in (
    ('fao_emi_total', fao_emi_total),
    ('lak_emi_energy', lak_emi_energy),
    ('cmip_conc_co2', cmip_conc_co2),
    ('prm_emi', prm_emi), 
):
    obj.to_netcdf(f'xr/nc/{fpath}.nc', engine='h5netcdf')

/usr/lib/python3/dist-packages/h5netcdf/core.py:480: H5pyDeprecationWarning: other_ds.dims.create_scale(ds, name) is deprecated. Use ds.make_scale(name) instead.
  h5ds.dims.create_scale(h5ds, scale_name)


In [25]:
for key, ds in primap_sources.items():
    ds.to_netcdf(f'xr/nc/primap_sources.nc', mode='a', group=key, engine='h5netcdf', encoding={var: {'compression': 'lzf'} for var in ds.keys()})

In [27]:
!du -sh xr/nc/*

764K	xr/nc/cmip_conc_co2.nc
7,5M	xr/nc/fao_emi_total.nc
36K	xr/nc/lak_emi_energy.nc
111M	xr/nc/primap_sources.nc
137M	xr/nc/prm_emi.nc


In [30]:
fao_emi_total_r = xr.open_dataset('xr/nc/fao_emi_total.nc')
lak_emi_energy_r = xr.open_dataset('xr/nc/lak_emi_energy.nc')
cmip_conc_co2_r = xr.open_dataset('xr/nc/cmip_conc_co2.nc')
prm_emi_r = xr.open_dataset('xr/nc/prm_emi.nc')

/home/pflueger/.local/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/usr/lib/python3/dist-packages/numpy/core/_asarray.py:85: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)


In [37]:
import h5py
h5 = h5py.File('xr/nc/primap_sources.nc', 'r')
primap_sources_r = {}
for key in h5.keys():
    primap_sources_r[key] = xr.open_dataset('xr/nc/primap_sources.nc', group=key, engine='h5netcdf')

# interactive analysis and plotting
* add / subtract / divide / multiply compatible, overlapping data sets
* timeit
* plot a historical time series, and something akin to gapminder (countries on a gdp vs emissions plot)
* select datasets which contain data on the N2O emissions of Finland
* in a selected dataset, select years in which the N2O emissions of Finland were above the 1950-2000 average
* resample non-yearly dataset to yearly dataset
* convert a dataset to tidy format

## aligned arithmetic

In [80]:
# extract two comptaible, partly overlapping data sets from primap-hist
a = prm_emi.loc[{'Category': ['IPC1'], 'Date': slice('1900', '1990'), 'Entity': ['CO2'], 'Scenario': ['HISTCR']}]
# NEED to use list of labels for aligned arithmetic! A single label gets converted into a scalar axis, which gets dropped in arithmetic.

In [85]:
# alternative spelling
a = prm_emi.sel(Category=['IPC1'], Date=slice('1900', '1990'), Entity=['CO2'], Scenario=['HISTCR'])
a

<xarray.DataArray 'Emissions' (Area: 216, Category: 1, Date: 91, Entity: 1, Scenario: 1)>
array([[[[[4.46e+02]],

         [[4.67e+02]],

         [[4.91e+02]],

         ...,

         [[6.12e+02]],

         [[6.49e+02]],

         [[1.64e+03]]]],



       [[[[1.70e+00]],

         [[1.79e+00]],

...

         [[2.41e+03]],

         [[2.23e+03]]]],



       [[[[4.62e+01]],

         [[4.82e+01]],

         [[4.75e+01]],

         ...,

         [[1.36e+04]],

         [[1.37e+04]],

         [[1.32e+04]]]]])
Coordinates:
  * Area      (Area) object 'ABW' 'AFG' 'AGO' 'AIA' ... 'YEM' 'ZAF' 'ZMB' 'ZWE'
  * Category  (Category) object 'IPC1'
  * Date      (Date) datetime64[ns] 1900-01-01 1901-01-01 ... 1990-01-01
  * Entity    (Entity) object 'CO2'
  * Scenario  (Scenario) object 'HISTCR'
Attributes:
    units:    Gg

In [86]:
b = prm_emi.loc[{'Area': ['DEU', 'FIN', 'ZWE']}]
b

<xarray.DataArray 'Emissions' (Area: 3, Category: 19, Date: 168, Entity: 13, Scenario: 2)>
array([[[[[4.20e+02, 4.45e+02],
          [1.48e+04, 1.52e+04],
          [     nan,      nan],
          ...,
          [     nan,      nan],
          [     nan,      nan],
          [     nan,      nan]],

         [[4.21e+02, 4.46e+02],
          [1.65e+04, 1.69e+04],
          [     nan,      nan],
          ...,
          [     nan,      nan],
          [     nan,      nan],
          [     nan,      nan]],

         [[4.24e+02, 4.49e+02],
          [1.88e+04, 1.92e+04],
          [     nan,      nan],
          ...,
...
          ...,
          [     nan,      nan],
          [     nan,      nan],
          [     nan,      nan]],

         [[3.78e+01, 3.78e+01],
          [     nan,      nan],
          [     nan,      nan],
          ...,
          [     nan,      nan],
          [     nan,      nan],
          [     nan,      nan]],

         [[5.57e+01, 5.57e+01],
          [     nan,      nan],
          [     nan,      nan],
          ...,
          [     nan,      nan],
          [     nan,      nan],
          [     nan,      nan]]]]])
Coordinates:
  * Area      (Area) object 'DEU' 'FIN' 'ZWE'
  * Category  (Category) object 'IPC1' 'IPC1A' 'IPC1B' ... 'IPCMAG' 'IPCMAGELV'
  * Date      (Date) datetime64[ns] 1850-01-01 1851-01-01 ... 2017-01-01
  * Entity    (Entity) object 'CH4' 'CO2' ... 'PFCSAR4 (CO2eq)' 'SF6'
  * Scenario  (Scenario) object 'HISTCR' 'HISTTP'
Attributes:
    units:    Gg

In [106]:
c = a + b

In [107]:
aq = a.pint.quantify(unit_registry=u)
bq = b.pint.quantify(unit_registry=u)

In [108]:
cq = aq + bq

In [109]:
bqt = bq.pint.to('metric_ton')

In [110]:
cqq = aq + bqt

In [112]:
a + b

<xarray.DataArray 'Emissions' (Area: 3, Category: 1, Date: 91, Entity: 1, Scenario: 1)>
array([[[[[6.400e+05]],

         [[6.380e+05]],

         [[6.260e+05]],

         [[6.740e+05]],

         [[6.980e+05]],

         [[7.500e+05]],

         [[7.100e+05]],

         [[8.660e+05]],

         [[8.760e+05]],

         [[8.740e+05]],

...

         [[1.570e+04]],

         [[1.484e+04]],

         [[1.766e+04]],

         [[1.656e+04]],

         [[1.700e+04]],

         [[2.280e+04]],

         [[2.600e+04]],

         [[2.720e+04]],

         [[2.740e+04]],

         [[2.640e+04]]]]])
Coordinates:
  * Area      (Area) object 'DEU' 'FIN' 'ZWE'
  * Category  (Category) object 'IPC1'
  * Date      (Date) datetime64[ns] 1900-01-01 1901-01-01 ... 1990-01-01
  * Entity    (Entity) object 'CO2'
  * Scenario  (Scenario) object 'HISTCR'

In [113]:
a - b

<xarray.DataArray 'Emissions' (Area: 3, Category: 1, Date: 91, Entity: 1, Scenario: 1)>
array([[[[[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

...

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]],

         [[0.]]]]])
Coordinates:
  * Area      (Area) object 'DEU' 'FIN' 'ZWE'
  * Category  (Category) object 'IPC1'
  * Date      (Date) datetime64[ns] 1900-01-01 1901-01-01 ... 1990-01-01
  * Entity    (Entity) object 'CO2'
  * Scenario  (Scenario) object 'HISTCR'

In [114]:
a * b

<xarray.DataArray 'Emissions' (Area: 3, Category: 1, Date: 91, Entity: 1, Scenario: 1)>
array([[[[[1.02400e+11]],

         [[1.01761e+11]],

         [[9.79690e+10]],

         [[1.13569e+11]],

         [[1.21801e+11]],

         [[1.40625e+11]],

         [[1.26025e+11]],

         [[1.87489e+11]],

         [[1.91844e+11]],

         [[1.90969e+11]],

...

         [[6.16225e+07]],

         [[5.50564e+07]],

         [[7.79689e+07]],

         [[6.85584e+07]],

         [[7.22500e+07]],

         [[1.29960e+08]],

         [[1.69000e+08]],

         [[1.84960e+08]],

         [[1.87690e+08]],

         [[1.74240e+08]]]]])
Coordinates:
  * Area      (Area) object 'DEU' 'FIN' 'ZWE'
  * Category  (Category) object 'IPC1'
  * Date      (Date) datetime64[ns] 1900-01-01 1901-01-01 ... 1990-01-01
  * Entity    (Entity) object 'CO2'
  * Scenario  (Scenario) object 'HISTCR'

In [116]:
a / b

<xarray.DataArray 'Emissions' (Area: 3, Category: 1, Date: 91, Entity: 1, Scenario: 1)>
array([[[[[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

...

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]],

         [[1.]]]]])
Coordinates:
  * Area      (Area) object 'DEU' 'FIN' 'ZWE'
  * Category  (Category) object 'IPC1'
  * Date      (Date) datetime64[ns] 1900-01-01 1901-01-01 ... 1990-01-01
  * Entity    (Entity) object 'CO2'
  * Scenario  (Scenario) object 'HISTCR'

## plotting

In [ ]:
plt.figure()
sel = prm_emi.xs(('DEU', 'IPCM0EL', 'KYOTOGHG (CO2eq)', 'HISTCR'), level=('Area', 'Category', 'Entity', 'Scenario'))
#sel = prm_emi.loc['DEU', 'IPCM0EL', :, 'KYOTOGHG (CO2eq)', 'HISTCR']
sel.pint.magnitude.plot()

In [118]:
prm_emi.loc[{'Area': 'DEU', 'Category': 'IPCM0EL', 'Entity': 'KYOTOGHG (CO2eq)', 'Scenario': 'HISTCR'}].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
[x for x in primap_sources if 'POP' in primap_sources[x]]

['UN2017P', 'UN2019P', 'MPD2018P', 'PMHSOCIOECO12']

In [149]:
pop_2000 = primap_sources['UN2019P']['POP'].loc[{'Date': '2000-01-01', 'Scenario': 'HISTORY'}].drop_vars(['Category', 'Class', 'Type', 'Scenario'])

In [153]:
emi_2000 = (prm_emi.loc[{'Category': 'IPCM0EL', 'Entity': 'KYOTOGHG (CO2eq)', 'Scenario': 'HISTCR', 'Date': '2000-01-01'}]
                   .drop_vars(['Category', 'Entity', 'Scenario'])
           )

In [168]:
fig, ax = plt.subplots()
comb_2000 = xr.Dataset({'POP': pop_2000, 'EMI': emi_2000.pint.quantify(unit_registry=u).pint.to('Tg').pint.dequantify()})
comb_2000.plot.scatter(x='POP', y='EMI', hue='Area')
ax.set_xlabel('Population in 2000')
ax.set_ylabel('Emissions in 2000 (CO2eq) / Tg')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Emissions in 2000 (CO2eq) / Tg')

## Selecting datasets and data
* select datasets which contain data on the N2O emissions of Finland
* in a selected dataset, select years in which the N2O emissions of Finland were above the 1950-2000 average

In [181]:
for source, ds in primap_sources.items():
    if 'N2O' not in ds:
        continue
    if 'FIN' not in ds['Area'] or 'IPC0' not in ds['Category']:
        continue
    if not np.all(np.isnan(ds['N2O'].loc[{'Area': 'FIN', 'Category': 'IPC0'}])):
        print(source)

CRF2020
EDGAR42COMPI
EDGAR50I
CRF2019
PRIMAPHIST20
EDGAR432I


In [188]:
da = primap_sources['PRIMAPHIST20']['N2O'].loc[{'Area': 'FIN', 'Category': 'IPC0', 'Scenario': 'HISTTP'}].squeeze()

In [189]:
avg2hXX = da.loc[{'Date': slice('1950', '2000')}].mean().item()
avg2hXX

22.8155464621459

In [190]:
da.loc[da > avg2hXX]

<xarray.DataArray 'N2O' (Date: 50)>
array([22.93627986, 23.50504739, 23.72240281, 23.8862899 , 24.26609438,
       25.07031252, 26.52174921, 27.1659699 , 26.87486302, 24.48126745,
       23.78169151, 25.34065936, 26.24261013, 26.63644459, 25.21584013,
       25.50779987, 26.81153158, 26.36323848, 25.43555587, 25.64939032,
       25.23117605, 24.96633261, 26.25516033, 26.64164902, 25.52220393,
       22.92523361, 23.48831524, 24.24063538, 24.60235597, 24.82996874,
       25.08372864, 23.94181711, 23.68743826, 24.30815136, 25.42688588,
       26.43740205, 27.02517848, 26.81678103, 26.36928441, 26.58584642,
       27.62953603, 27.20736253, 24.21194636, 27.02294059, 24.71151892,
       25.19164292, 26.698377  , 26.45006638, 27.5693746 , 26.67504966])
Coordinates:
    Category  <U10 'IPC0'
    Class     <U5 'TOTAL'
    Type      <U3 'NET'
    Scenario  <U6 'HISTTP'
    Area      <U9 'FIN'
  * Date      (Date) datetime64[ns] 1967-01-01 1968-01-01 ... 2016-01-01
Attributes:
    units:    gigagram

## Resampling
* resample non-yearly dataset to yearly dataset

In [191]:
cmip_conc_co2

<xarray.DataArray 'CO2 concentration' (Area: 3, Date: 24180)>
array([[0.00027837, 0.00027871, 0.00027901, ..., 0.00039667, 0.00039819,
        0.00039918],
       [0.00027905, 0.00027967, 0.00028027, ..., 0.00039746, 0.00040032,
        0.0004022 ],
       [0.00027769, 0.00027776, 0.00027776, ..., 0.00039588, 0.00039606,
        0.00039616]])
Coordinates:
  * Area     (Area) object 'global' 'nh' 'sh'
  * Date     (Date) object -001-01-01 00:00:00 ... 2014-12-01 00:00:00
Attributes:
    units:    dimensionless

In [212]:
cmip_conc_co2.groupby('Date.year').mean()

<xarray.DataArray 'CO2 concentration' (Area: 3, year: 2015)>
array([[0.00027745, 0.00027714, 0.00027716, ..., 0.00039302, 0.00039572,
        0.00039755],
       [0.00027745, 0.00027714, 0.00027716, ..., 0.00039504, 0.00039771,
        0.00039959],
       [0.00027745, 0.00027714, 0.00027716, ..., 0.000391  , 0.00039374,
        0.0003955 ]])
Coordinates:
  * Area     (Area) object 'global' 'nh' 'sh'
  * year     (year) int64 -1 1 2 3 4 5 6 ... 2008 2009 2010 2011 2012 2013 2014

## Tidy data
convert to tidy dataformat

In [214]:
prm_emi.to_dataframe()

Emissions
Area Category  Date       Entity          Scenario           
ABW  IPC1      1850-01-01 CH4             HISTCR     0.000387
                                          HISTTP     0.000387
                          CO2             HISTCR    40.300000
                                          HISTTP    40.300000
                          FGASES (CO2eq)  HISTCR          NaN
...                                                       ...
ZWE  IPCMAGELV 2017-01-01 PFCS (CO2eq)    HISTTP          NaN
                          PFCSAR4 (CO2eq) HISTCR          NaN
                                          HISTTP          NaN
                          SF6             HISTCR          NaN
                                          HISTTP          NaN

[17926272 rows x 1 columns]

# Interpolate
* delete some data points from a data set and fill them in using linear interpolation

In [204]:
da = primap_sources['PRIMAPHIST20']['N2O'].loc[{'Area': 'FIN', 'Scenario': 'HISTTP', 'Category': 'IPC0'}].squeeze()

In [207]:
da.loc[{'Date': slice('1990', '1994')}] = np.nan

In [208]:
plt.figure()
da.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [211]:
plt.figure()
da.interpolate_na('Date').plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …